# Код для парсинга профилей и постов LinkedIn

In [1]:
import time
import configparser
import random
import re
import os.path

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

**Загружаем конфиг**

In [10]:
# путь к файлу расширения для Chrome "Доступ к LinkedIn"
EXTENSION_PATH = '1.5_0.crx'

# конфиг
CFG_FILE = 'parser.ini'

"""
файл конфигурации необходимо предварительно создать,
формат файла parser.ini:
[LINKEDIN]
USER_LOGIN = эл_почта_без_кавычек
USER_PASSWORD = пароль
""";

# загружаем данные из конфига
conf = configparser.ConfigParser()
try:
    conf.read(CFG_FILE)
    USER_LOGIN = conf['LINKEDIN']['USER_LOGIN']
    USER_PASSWORD = conf['LINKEDIN']['USER_PASSWORD']
except:
    print(f'Не удалось прочитать файл конфигурации: {CFG_FILE}')
    quit() # нужен только при переносе кода в .py

**Общие процедуры и функции**

In [46]:
# прокрутка страницы, для подгрузки динамического контента
def get_scrolled_page(driver, num_scrolls=15, pause_time=0.5):
    """
    Функция прокручивает страницу, загруженную в экземпляр driver,
    num_scrolls раз, с pause_time паузами между прокрутками.
    Возвращает код страницы.
    """
    # текущая высота body
    last_height = driver.execute_script('return document.body.scrollHeight')
    for i in range(num_scrolls):
        
        # нажимаем кнопку PageDown 5 раз
        for _ in range(5):
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
            # делаем паузу для загрузки динамического контента
            time.sleep(random.uniform(pause_time, 3))
        
        # вычисляем новую высоту body
        new_height = driver.execute_script('return document.body.scrollHeight')
        if new_height == last_height:
            break 
        last_height = new_height

    return driver

In [4]:
# формируем запрос на поиск людей по ключевым словам
def search_people_url(keywords, tags, page_num=1):
    """
    Функция на вход получает ключевые слова,
    список тем публикаций для поиска и номер страницы.
    Возвращает url для запроса страницы.
    """
    # преобразуем теги в строку
    tags_str = str(tags).replace(" ", "").replace("'", '"')
    
    # формируем строку запроса
    search_url = 'https://www.linkedin.com/search/results/people/'
    search_url += f'?keywords={keywords}'
    search_url += '&origin=FACETED_SEARCH'
    search_url += f'&page={page_num}'
    search_url += '&profileLanguage=["ru"]'
    # темы публикаций (хештеги)
    search_url += f'&talksAbout={tags_str}'
    
    return search_url

In [5]:
# получаем список профилей на странице
def get_profiles(driver):
    """
    Функция получает драйвер открытой страницы,
    ищет ссылки на доступные профили пользователей и возвращает
    список id пользователей.
    """
    # список найденных профилей
    profiles = []

    # ищем на странице ссылки на профили
    finded_profiles = driver.find_elements(
        By.CSS_SELECTOR, "span.entity-result__title-text a.app-aware-link"
    )
    for profile in finded_profiles:
        # получаем url на профиль пользователя
        url = profile.get_attribute("href")
        # если url ссылается на доступный профиль
        if 'linkedin.com/in' in url:
            # оставляем только id профиля
            profile_id = url.split('?')[0].split('/in/')[1]
            # добавляем id в список
            profiles.append(profile_id)

    # избавляемся от дублей, если вдруг появятся
    profiles = list(set(profiles))
    return profiles

In [6]:
# собираем информацию о пользователе
def get_user_info(driver, user_id):
    """
    Функция парсит со страницы профиля информацию о пользователе.
    На вход получает, драйвер и идентификатор пользователя.
    На выходе возвращает список с данным профиля
    """
    # прокручиваем страницу до конца что бы подгрузился динамический контент
    driver = get_scrolled_page(driver, num_scrolls=3, pause_time=0.5)

    # извлекаем код страницы
    src = driver.page_source

    # предеаем код страницы в парсер
    soup = BeautifulSoup(src, 'lxml')    
    
    # извлекаем HTML содержаший имя и заголовок
    intro = soup.find('div', {'class': 'mt2 relative'})

    # получаем имя
    user_name = ''
    try:
        name_loc = intro.find("h1")
        user_name = name_loc.get_text().strip()
    except: ...

    # заголовок, обычно тут пишут где работает или специальность или навыки
    user_head = ''
    try:
        head_at_loc = intro.find("div", {'class': 'text-body-medium'})
        user_head = head_at_loc.get_text().strip()
    except: ...

    # получаем теги
    user_tags = ''
    try:
        # темы публикаций
        tags_at_loc = intro.find("div", {'class': 'text-body-small t-black--light break-words mt2'})
        # уточняем 
        tags_at_loc = tags_at_loc.find('span', {'aria-hidden': 'true'})
        # убираем лишние символы
        user_tags = tags_at_loc.get_text().split(':')[1].strip()
        user_tags = user_tags.replace('#','').replace(' и',',')
    except: ...

    # получаем локацию пользователя
    user_location = ''
    try:
        location_at_loc = intro.find("div", {'class': 'pv-text-details__left-panel mt2'})
        # уточняем
        location_at_loc = location_at_loc.find('span', {'class': 'text-body-small'})
        user_location = location_at_loc.get_text().strip()
    except: ...

    # место работы
    user_work = ''
    try:
        work_at_loc = intro.find("div", {'class': 'inline-show-more-text'})
        user_work = work_at_loc.get_text().strip()
    except: ...

    # количество отслеживающих и контактов
    user_viewwers, user_contacts = '0', '0'
    try:
        stat_at_loc = soup.find("ul", {'class': 'pv-top-card--list pv-top-card--list-bullet'})
        user_viewwers = stat_at_loc.find_all("span")[0].get_text().strip()
        user_contacts = stat_at_loc.find_all("span")[2].get_text().strip()
    except: ...

    # общие сведения
    user_common_info = ''
    try:
        common_at_loc = soup.find("div", {'class': 'display-flex ph5 pv3'})
        user_common_info = common_at_loc.find_all('span')[0].get_text().strip()
    except: ...

    # должность
    user_position = ''
    try:
        position_at_loc = soup.find("ul", {'class': 'pvs-list'})
        user_position = position_at_loc.find_all('span')[0].get_text().strip()
    except: ...
        
    return [
        user_name, user_head, user_work, user_position, user_tags,
        user_location, user_viewwers, user_contacts, user_common_info
    ]

In [7]:
# парсим данные публикации
def get_post_info(post):
    """
    Функция на вход получает блок кода с публикацией.
    Возвращает список параметров публикации: текст и реакции.
    """
    # текст поста
    post_text = 'no text'
    try:
        post_text = post.find('span', {'class': 'break-words'}).get_text().strip()
    except: ...

    # блок реакций на пост
    likes, comments, reposts = '0', '0', '0'
    try:
        reaсtions = post.find('ul', {'class': 'social-details-social-counts'})
        try:
            likes = reaсtions.find(
                'span', {'class': 'social-details-social-counts__reactions-count'}
            ).get_text().strip().replace('\xa0', ' ')
            
        except: ...
        try:
            comments = reaсtions.find(
                'li', {'class': 'social-details-social-counts__comments'}
            ).get_text().strip().replace('\xa0', ' ')
            comments = re.match('^[\d]+', comments)[0]
        except: ...
        try:
            reposts = reaсtions.find(
                'li', {'class': 'social-details-social-counts__item social-details-social-counts__item--with-social-proof'}
            ).get_text().strip().replace('\xa0', ' ')
            reposts = re.match('^[\d]+', reposts)[0]
        except: ...
    except: ...
        
    return [post_text, likes, comments, reposts]

**Создаем и запускаем браузер**

In [39]:
# подключаем расширение к драйверу
options = webdriver.ChromeOptions()
options.add_extension(EXTENSION_PATH)

# меняем стратегию - ждать, пока свойство document.readyState примет значение interactive
options.page_load_strategy = 'eager'

# запускаем Chrome с расширением
driver = webdriver.Chrome(options=options)

**Вход в LinkedIn**

In [11]:
# открываем страницу входа linkedIn, необходимо отключить двухфакторную аутонтификацию
driver.get("https://linkedin.com/uas/login")

# ожидаем загрузку страницы
time.sleep(4)

# поле ввода имени пользователя
username = driver.find_element(By.ID, "username")

# вводим свой Email
username.send_keys(USER_LOGIN)

# поле ввода пароля
pword = driver.find_element(By.ID, "password")

# вводим пароль
pword.send_keys(USER_PASSWORD)

# нажимаем кнопку Войти
# Формат (синтаксис) написания XPath --> //tagname[@attribute='value']
driver.find_element(By.XPATH, "//button[@type='submit']").click()

**Параметры парсинга**

In [9]:
# теги, темы публикаций

#KEYWORDS = 'разработка по'
#TAGS = ['softwaredevelopment', 'webdevelopment', 'startup', 'it', 'design']
#CSV_FILE_NAME = 'profiles_id_1.csv'

#KEYWORDS = 'devops'
#TAGS = ['devops', 'aws', 'python', 'cloud', 'kubernetes']
#CSV_FILE_NAME = 'profiles_id_2.csv'

#KEYWORDS = 'data science'
#TAGS = ['datascience', 'machinelearning', 'ai', 'artificialintelligence', 'dataanalytics']
#CSV_FILE_NAME = 'profiles_id_3.csv'

#KEYWORDS = 'project management'
#TAGS = ['projectmanagement', 'business', 'agile', 'scrum', 'it']
#CSV_FILE_NAME = 'profiles_id_4.csv'

KEYWORDS = 'design ui ux'
TAGS = ['design', 'webdesign', 'ux', 'ui', 'uxdesign', 'uidesign']
CSV_FILE_NAME = 'profiles_id_5.csv'

**Собираем ID пользователей**

In [10]:
"""
# число страниц для парсинга, в бесплатном аккаунте доступно не более 100
NUM_PAGES = 100

# пустой датафрейм для id пользователей
df = pd.DataFrame(columns=['id'])

for page_num in range(1, NUM_PAGES+1):
    
    # выводим номер страницы, в случае сбоя можно будет начать новый парсинг с нее
    print(page_num, end='  ')
    
    # формируем url запроса
    people_url = search_people_url(KEYWORDS, TAGS, page_num=page_num)
    
    # запрашиваем и открываем страницу
    driver.get(people_url)
    
    # получаем и добавляем список найденных id профилей на странице
    profiles_id = get_profiles(driver)
    
    # добавляем данные в датафрейм
    df = pd.concat(
        [df, pd.DataFrame({'id': profiles_id})]
    ).reset_index(drop=True)
    
    # сохраняем в CSV
    df.to_csv(CSV_FILE_NAME)

    time.sleep(random.uniform(3, 5))
""";

**Собираем все id в один датафрейм**

In [11]:
# имя файла для сохранения профилей юзеров
CSV_PROFILES_FILE_NAME = 'profiles.csv'

# названия столбцов для хранения данных о пользователях
profile_columns = [
    'user_name', # имя
    'user_head', # заголовок
    'user_work', # последннее/текущее место работы
    'user_position', # должность
    'user_tags', # теги, интересы
    'user_location', # адрес
    'user_viewers', # число подписчиков
    'user_contacts', # число контактов
    'user_common_info' # общая информация
]

In [12]:
# если файл с профилями уже существует
if os.path.exists(CSV_PROFILES_FILE_NAME):
    # загружаем датафрейм из файла
    df = pd.read_csv(CSV_PROFILES_FILE_NAME, index_col=0)
else:
    # список файлов c id пользователей
    list_csv_files = [
        'profiles_id_1.csv',
        'profiles_id_2.csv',
        'profiles_id_3.csv',
        'profiles_id_4.csv',
        'profiles_id_5.csv',
    ]
    # пустой DF
    df = pd.DataFrame(columns=['id'])

    # соберем все файлы в один DF
    for csv_file in list_csv_files:
        df = pd.concat(
            [df, pd.read_csv(csv_file, index_col=0)]
        ).reset_index(drop=True)

    # удаляем дубли
    df = df.drop_duplicates()

    df = df.reindex(columns = df.columns.tolist() + profile_columns)

print('Всего профилей:', len(df))

Всего профилей: 1709


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1709 entries, 0 to 1864
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                1709 non-null   object
 1   user_name         299 non-null    object
 2   user_head         299 non-null    object
 3   user_work         278 non-null    object
 4   user_position     299 non-null    object
 5   user_tags         108 non-null    object
 6   user_location     297 non-null    object
 7   user_viewers      299 non-null    object
 8   user_contacts     299 non-null    object
 9   user_common_info  277 non-null    object
dtypes: object(10)
memory usage: 146.9+ KB


**Парсим профили и посты**

In [14]:
# имя файла для сохранения публикаций
CSV_POSTS_FILE_NAME = 'posts.csv'

# названия столбцов для хранения публикаций
posts_columns = [
    'user_id', # id профиля
    'text', # текст публикации
    'likes', # количество реакций
    'comments', # количество комментариев
    'reposts', # количество комментариев
]

In [15]:
# если файл с профилями уже существует
if os.path.exists(CSV_POSTS_FILE_NAME):
    # загружаем датафрейм из файла
    df_posts = pd.read_csv(CSV_POSTS_FILE_NAME, index_col=0)
else:
    # пустой датафрейм для текстов публикаций
    df_posts = pd.DataFrame(columns=posts_columns)

In [16]:
df_posts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6177 entries, 0 to 6176
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   6177 non-null   object
 1   text      6177 non-null   object
 2   likes     6177 non-null   object
 3   comments  6177 non-null   int64 
 4   reposts   6177 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 289.5+ KB


In [17]:
# с какого профиля стартуем
start_idx = df.user_name.nunique()
start_idx

299

In [20]:
# парсим данные из профилей
for profile_id in df.id[start_idx:]:
    
    # для контроля выводим на экран текущий ID профиля
    print(profile_id)
    
    # получаем url профиля пользователя
    profile_url = f'https://www.linkedin.com/in/{profile_id}/'

    # открываем ссылку profile_url
    driver.get(profile_url)

    # парсим информацию профиля
    user_info = get_user_info(driver, profile_id)
    
    # сохраняем данные в датафрейм
    df.loc[df.id == profile_id, profile_columns] = user_info
    
    # сохраняем в CSV
    df.to_csv(CSV_PROFILES_FILE_NAME)
    
    # пауза
    time.sleep(random.uniform(10, 20))
    
    # URL на все публикации пользователя
    posts_url = f'https://www.linkedin.com/in/{profile_id}/recent-activity/all/'

    driver.get(posts_url)

    # получаем код проскроленой страницы
    src = get_scrolled_page(driver, num_scrolls=25, pause_time=0.5).page_source

    # передаем код страницы в парсер
    soup = BeautifulSoup(src, 'lxml')

    # получаем список постов
    posts_block = soup.find_all(
        'li', {'class': 'profile-creator-shared-feed-update__container'}
    )

    print(f'posts: {len(posts_block)}')

    count_posts = 1
    
    for post in posts_block:
        
        # номер поста для контроля
        print(count_posts, end=' ')
        count_posts += 1
        
        # получаем данные публикации
        post_info = get_post_info(post)
        
        if not post_info[0] == 'no text':
            # добавляем данные в датафрейм
            df_posts.loc[len(df_posts.index)] = [profile_id] + post_info
        
        # сохраняем в CSV
        df_posts.to_csv(CSV_POSTS_FILE_NAME)
    
    print()

antonlenskiy
posts: 129
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 
yersar
posts: 33
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 
katerynaovchar
posts: 40
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 
bohdanstruk
posts: 5
1 2 3 4 5 
yuriy-pryima
posts: 40
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 
valeryiayatsuk
posts: 40
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 
haideraliedu
posts

posts: 40
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 
andranik-mkhitaryan
posts: 39
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 
andranikmirzoyan
posts: 40
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 
kamil-ismagilov-react
posts: 10
1 2 3 4 5 6 7 8 9 10 
aydesign1
posts: 9
1 2 3 4 5 6 7 8 9 
andrey-davidovich
posts: 0

kamran-yakubov
posts: 39
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 
auphotoart
posts: 24
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 
shakhobiddin-giyasov-761a85254
posts: 4
1 2 3 4 
suxrobgm
posts: 39
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 
katerina-shalova-090b3915b
posts: 0

anastasia-prokhorova-a1b141119
posts: 15
1 2 3 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"body.render-mode-BIGPIPE"}
  (Session info: chrome=114.0.5735.134); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x010FA813+48355]
	(No symbol) [0x0108C4B1]
	(No symbol) [0x00F95358]
	(No symbol) [0x00FC09A5]
	(No symbol) [0x00FC0B3B]
	(No symbol) [0x00FEE232]
	(No symbol) [0x00FDA784]
	(No symbol) [0x00FEC922]
	(No symbol) [0x00FDA536]
	(No symbol) [0x00FB82DC]
	(No symbol) [0x00FB93DD]
	GetHandleVerifier [0x0135AABD+2539405]
	GetHandleVerifier [0x0139A78F+2800735]
	GetHandleVerifier [0x0139456C+2775612]
	GetHandleVerifier [0x011851E0+616112]
	(No symbol) [0x01095F8C]
	(No symbol) [0x01092328]
	(No symbol) [0x0109240B]
	(No symbol) [0x01084FF7]
	BaseThreadInitThunk [0x769D7D59+25]
	RtlInitializeExceptionChain [0x7788B74B+107]
	RtlCle

In [49]:
# закрываем браузер
driver.quit()

**Результат**

In [22]:
# профили
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1709 entries, 0 to 1864
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                1709 non-null   object
 1   user_name         398 non-null    object
 2   user_head         398 non-null    object
 3   user_work         377 non-null    object
 4   user_position     398 non-null    object
 5   user_tags         207 non-null    object
 6   user_location     396 non-null    object
 7   user_viewers      398 non-null    object
 8   user_contacts     398 non-null    object
 9   user_common_info  376 non-null    object
dtypes: object(10)
memory usage: 146.9+ KB


In [23]:
# публикации
df_posts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8610 entries, 0 to 8609
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   8610 non-null   object
 1   text      8610 non-null   object
 2   likes     8610 non-null   object
 3   comments  8610 non-null   object
 4   reposts   8610 non-null   object
dtypes: object(5)
memory usage: 403.6+ KB
